This script is only for using in the CSC environment.

In [1]:
!which python

/projappl/project_2013104/pengyan1/venv/bin/python


In [2]:
import sys
import os

folder_path = os.path.abspath("/projappl/project_2013104/pengyan1/venv/lib/python3.10/site-packages")
if folder_path not in sys.path:
    sys.path.append(folder_path)

sys.path

['/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python310.zip',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/lib-dynload',
 '',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages',
 '/projappl/project_2013104/pengyan1/venv/lib/python3.10/site-packages']

In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MLP, PointNetConv, fps, global_max_pool, radius
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, knn_interpolate
from torch_geometric.typing import WITH_TORCH_CLUSTER
from torch_geometric.data import Data

import open3d as o3d
import numpy as np

from sklearn.linear_model import RANSACRegressor
from scipy.spatial import ConvexHull

if not WITH_TORCH_CLUSTER:
    quit("This example requires 'torch-cluster'")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
class SAModule(torch.nn.Module):
    def __init__(self, ratio, r, nn):
        super().__init__()
        self.ratio = ratio
        self.r = r
        self.conv = PointNetConv(nn, add_self_loops=False)

    def forward(self, x, pos, batch):
        idx = fps(pos, batch, ratio=self.ratio)
        row, col = radius(pos, pos[idx], self.r, batch, batch[idx],
                          max_num_neighbors=64)
        edge_index = torch.stack([col, row], dim=0)
        x_dst = None if x is None else x[idx]
        x = self.conv((x, x_dst), (pos, pos[idx]), edge_index)
        pos, batch = pos[idx], batch[idx]
        return x, pos, batch


class GlobalSAModule(torch.nn.Module):
    def __init__(self, nn):
        super().__init__()
        self.nn = nn

    def forward(self, x, pos, batch):
        x = self.nn(torch.cat([x, pos], dim=1))
        x = global_max_pool(x, batch)
        pos = pos.new_zeros((x.size(0), 3))
        batch = torch.arange(x.size(0), device=batch.device)
        return x, pos, batch

In [5]:
class FPModule(torch.nn.Module):
    def __init__(self, k, nn):
        super().__init__()
        self.k = k
        self.nn = nn

    def forward(self, x, pos, batch, x_skip, pos_skip, batch_skip):
        x = knn_interpolate(x, pos, pos_skip, batch, batch_skip, k=self.k)
        if x_skip is not None:
            x = torch.cat([x, x_skip], dim=1)
        x = self.nn(x)
        return x, pos_skip, batch_skip

In [6]:
class Net(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # Input channels account for both `pos` and node features.
        self.sa1_module = SAModule(0.2, 0.2, MLP([3 + 6, 64, 64, 128])) # 3 (pos) + 6 (x)
        self.sa2_module = SAModule(0.25, 0.4, MLP([128 + 3, 128, 128, 256]))
        self.sa3_module = GlobalSAModule(MLP([256 + 3, 256, 512, 1024]))

        self.fp3_module = FPModule(1, MLP([1024 + 256, 256, 256]))
        self.fp2_module = FPModule(3, MLP([256 + 128, 256, 128]))
        self.fp1_module = FPModule(3, MLP([128 + 6, 128, 128, 128]))

        self.mlp = MLP([128, 128, 128, num_classes], dropout=0.5, norm=None)

        self.lin1 = torch.nn.Linear(128, 128)
        self.lin2 = torch.nn.Linear(128, 128)
        self.lin3 = torch.nn.Linear(128, num_classes)

    def forward(self, data):
        sa0_out = (data.x, data.pos, data.batch)
        sa1_out = self.sa1_module(*sa0_out)
        sa2_out = self.sa2_module(*sa1_out)
        sa3_out = self.sa3_module(*sa2_out)

        fp3_out = self.fp3_module(*sa3_out, *sa2_out)
        fp2_out = self.fp2_module(*fp3_out, *sa1_out)
        x, _, _ = self.fp1_module(*fp2_out, *sa0_out)

        return self.mlp(x).log_softmax(dim=-1)


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = Net(num_classes=13).to(device)

cuda


In [8]:
# Load the checkpoint dictionary
checkpoint = torch.load('/scratch/project_2013104/checkpoints/pointnet2_s3dis_transform_seg_x6_45_checkpoint.pth', map_location=device)
# Extract the model state dictionary
model_state_dict = checkpoint['model_state_dict']

# print(model_state_dict)

In [9]:
model.load_state_dict(model_state_dict)
model.eval()

Net(
  (sa1_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(9, 64, 64, 128), global_nn=None)
  )
  (sa2_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(131, 128, 128, 256), global_nn=None)
  )
  (sa3_module): GlobalSAModule(
    (nn): MLP(259, 256, 512, 1024)
  )
  (fp3_module): FPModule(
    (nn): MLP(1280, 256, 256)
  )
  (fp2_module): FPModule(
    (nn): MLP(384, 256, 128)
  )
  (fp1_module): FPModule(
    (nn): MLP(134, 128, 128, 128)
  )
  (mlp): MLP(128, 128, 128, 13)
  (lin1): Linear(in_features=128, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=128, bias=True)
  (lin3): Linear(in_features=128, out_features=13, bias=True)
)

In [10]:
# Pcd files
#pcd_path = "/scratch/project_2013104/Smartlab-2024-04-05_10-58-26_colour_cleaned.pcd"
pcd_path = "/scratch/project_2013104/datasets/SmartLab_2024_E57_Single_5mm.pcd"
#pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/SmartLab_2024_E57_Single_5mm.ply"
pcd = o3d.io.read_point_cloud(pcd_path)

In [11]:
print(len(pcd.points))
print(len(pcd.colors))
print(len(pcd.normals))

35966772
35966772
0


In [12]:
def normalize_points_corner(points):
    # Step 1: Shift points so that the minimum x, y, z becomes the origin.
    min_vals = np.min(points, axis=0)
    shifted_points = points - min_vals  # Now the lower bound is (0,0,0)
    
    # Step 2: Compute the scaling factors from the shifted points.
    # The maximum after shifting represents the range in each dimension.
    max_vals = np.max(shifted_points, axis=0)
    scale = max_vals.copy()
    
    # Avoid division by zero if any dimension is flat.
    scale[scale == 0] = 1
    
    # Normalize the shifted points to the [0, 1] interval.
    normalized_points = shifted_points / scale

    return normalized_points

normalized = normalize_points_corner(np.array(pcd.points))

In [13]:
# Extract coordinates and colors from the point cloud
pcd_points = torch.tensor(np.array(pcd.points), dtype=torch.float32)  
pcd_colors = torch.tensor(np.array(pcd.colors), dtype=torch.float32)
pcd_normalized = torch.tensor(normalized, dtype=torch.float32)

In [14]:
# Concatenate coordinates and colors to form the input features
x = torch.cat([pcd_colors, pcd_normalized], dim=1) # 6 (x)

In [15]:
# Create a Data object with x (6 features) and pos (coordinates)
data = Data(x=x, pos=pcd_points)

#pre_transform = T.NormalizeScale()
#data = pre_transform(data)

In [16]:
# If you have only one point cloud
dataset = [data]  # List of Data objects

# Create a DataLoader (batch_size can be adjusted as needed)
custom_loader = DataLoader(dataset, batch_size=64, shuffle=False,
                         num_workers=10)

In [ ]:
import time
model.eval()
with torch.no_grad():
    start_time = time.time()
    for data in custom_loader:
        data = data.to(device)
        predictions = model(data)
        labels = predictions.argmax(dim=-1)
        # Process the labels as needed
        #labels_arr = labels.cpu().numpy()
        # Count occurrences of labels
        unique_labels, label_counts = torch.unique(labels, return_counts=True)
        # Combine and print
        #result_labels = np.array(list(zip(unique_labels, label_counts)))
        print("Label counts:")
        for label, count in zip(unique_labels, label_counts):
            print(f"Label {label.item()}: {count.item()}")
    end_time = time.time()
    print(f"Total inference time: {end_time - start_time:.4f} seconds")  


In [ ]:

# Define the number of classes in your model's predictions
num_classes = 13  # Adjust based on your number of classes


# Define a fixed color map for 13 labels
color_map = np.array([
    [1.0, 0.0, 0.0],  # Label 0: Red,  'ceiling'
    [0.0, 1.0, 0.0],  # Label 1: Green, 'floor'
    [0.0, 0.0, 1.0],  # Label 2: Blue,  'wall'
    [1.0, 1.0, 0.0],  # Label 3: Yellow, 'beam'
    [1.0, 0.0, 1.0],  # Label 4: Magenta, 'column'
    [0.0, 1.0, 1.0],  # Label 5: Cyan, 'window'
    [0.5, 0.5, 0.5],  # Label 6: Gray, 'door'
    [1.0, 0.5, 0.0],  # Label 7: Orange, 'chair'
    [0.5, 0.0, 1.0],  # Label 8: Purple, 'table'
    [0.5, 1.0, 0.5],  # Label 9: Light Green, 'bookcase'
    [0.5, 0.5, 1.0],  # Label 10: Light Blue, 'sofa'
    [1.0, 0.5, 0.5],  # Label 11: Pink, 'board'
    [0.0, 0.0, 0.0]   # Label 12: Black, 'clutter'
])

In [ ]:
# Assuming `labels` is the tensor containing predicted labels for each point
predicted_colors = color_map[labels.cpu().numpy()]  # Shape: [num_points, 3]


In [ ]:
print(len(predicted_colors))

In [ ]:
# Assuming `pcd` is your Open3D point cloud object
pcd.colors = o3d.utility.Vector3dVector(predicted_colors)

In [ ]:
# Visualize the point cloud with colored labels
o3d.visualization.draw_geometries([pcd])

In [ ]:
# Save the point cloud to a file
save_path = '/scratch/project_2013104/labelled/Smartlab_aalto_label_pointnet2_x6_ori_20250414.ply'
o3d.io.write_point_cloud(save_path, downpcd)
